In [37]:
import googletrans
from googletrans import Translator
import pandas as pd
import unicodedata
import time
import re
import math
from json import JSONDecodeError


def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')
    #replaces all accents with their unicode counterparts

def translateWord(word):
    translator = Translator()
    result=translator.translate(word,dest='en')
    #success=result.extra_data['translation'][0][4]
    #alternative='none'
    #if result.extra_data['possible-mistakes']!=None and strip_accents(result.text)==strip_accents(word):
    if strip_accents(result.text)==strip_accents(word):
        if result.extra_data['possible-mistakes']==None:
            new_word=word
            status=1
        else:
            alternative=result.extra_data['possible-mistakes'][1]
            result=translator.translate(alternative,dest='en')
            if result.extra_data['possible-mistakes']!=None:
                new_word=word
                status=0#it is not found
            else:
                new_word=result.text
                status=1
    else:
        new_word_proposed=result.text
        if strip_accents(new_word_proposed)!=strip_accents(word):#omitted symbol problem for vietnamstina
            new_word=result.text
            status=1
        else:#in case of no suggestions being offered from english, try either a loop or use expected languages
            #German translation loop, this catches the issue of words untranslated because e.g. language names are in the google db under english
            word_german=translator.translate(word,dest='de')
            word_english=translator.translate(word_german.text,dest='en')
            if word_english.text==word_german.text:
                new_word=word
                status=0
                new_word, status=tryLanguages(word)
                #USE e.g. CZECH
            else: 
                status=1
                new_word=word_english.text
    return new_word, status#, alternative
#maybe make separate functions of loop parts for readability and not rewriting the same stuff
#Remove the word language and spaces at the end

def tryLanguages(word):    
    status=0
    lang_list=['cs','es','ca','it','hi','id']#we could let this depend on the expected soscisurvey language
    for i in range(len(lang_list)):
        #print(i,lang_list[i])
        result=translator.translate(word,dest='en',src=lang_list[i])
        new_word=result.text
        if strip_accents(result.text)!=strip_accents(word):#not 100%fool proof!
            status=1
            #print(lang_list[i])
            break
        time.sleep(0.01)
    return new_word, status

def transParser(translation):
    temp = re.sub('Language$', '', translation)
    temp = re.sub('language$', '', temp)
    clean_translation=temp.rstrip()
    return clean_translation

In [31]:


def wrap(word):
    #print(word)
    #nan_check=isNaN(word)
    #if nan_check:
    if isinstance(word,str):
        while True:
            try:
                translation,stat=translateWord(word)
                ans=transParser(translation)
                break
            except JSONDecodeError:#check this, maybe use different entry points of translate
                print('JSON error')
                time.sleep(10)#wait 25 hours, or alter vpn?

                try:
    
                    translation=translateWord(word)
                    ans=transParser(translation)
                    break
                except JSONDecodeError:
                    print('final JSON error')
                    ans=word
                    break
            except OSError:
                time.sleep(1000)
                print('Connection issue')
                    
        print(ans)
        return ans.lower()
    else:
        print('NaN')
        return math.nan

In [64]:

df_translate=pd.read_excel('Countries_Citizenships.xlsx',sheet_name='Tabelle3')  
#list_rows=['D045_01','D046_01','D047_01','D049_01','D072_01','D075_01', 'D051_01', 'D052_01','D053_01','D055_06a','D055_07a','D055_08a','D059_06a','D059_07a','D059_08a'
#         ]
list_rows=['COUNTRIES_CITIZENSHIPS']
#len(list_rows)

for i in range(len(list_rows)):
    print(list_rows[i]+'_T')
    new_list_rows=list_rows[i]+'_T'
    #df_translate[new_list_rows]=df_translate[list_rows[i]]
    df_translate[new_list_rows] = df_translate[list_rows[i]].apply(lambda x: wrap(x))
    df_translate[new_list_rows] = df_translate[new_list_rows].str.capitalize()#to be tested
df2.to_excel('data_migration_translation_cc_070420.xlsx')
#Maybe only do this for surveys taken in non-english country

COUNTRIES_CITIZENSHIPS_T
Italy
United states of America
United kingdom
Germany
USA
Sweden
Netherlands
Australia
Singapore
United Kingdom
Thailand
Europe
Wherever there is a better work opportunity
Qatar
Cyprus
Japan
US,CANADA germany
France
Canada
US
More interesting job
higher income
interest in country
Environment reasons
Adventure
UAE
South Korea
Denmark
Singapore
america
England
Study and job purpose
Syrian
Syria
Syria
Saudi Arabia
Maldives
Saudi Arabia
Dubai
Spain
United States of America
Russia
U.S.A
UK
No idea yet
Khartoum American School, Khartoum, North Sudan
North Sudan
Finland
none
WK
Russia
New Zealand
United States
PARIS
America
Australia
Germany
Italy
India
Norway
Thailand,
Las Vegas
Japan, USA, Canada
London
UK
United States of America
USA, Germany, Japan
Seoul
Norway
U.S
U.K
Great Britain
Switzerland
london
Singapore
Britain
Ireland
France
US, California
Portugal
Study
Boston
Las Vegas
Bahrain
Germany,  France Or spain
United Arab Emirates
U.S.A.
USA, Canada, Japan
Stud

JSON error
final JSON error
Ethiopia
JSON error
final JSON error
no se ve la pregunta
JSON error
final JSON error
Kenya 
JSON error
final JSON error
Rusa
JSON error
final JSON error
No ha aparecido la pregunta
JSON error
final JSON error
País Bajo
JSON error
final JSON error
UK,
JSON error
final JSON error
Francia,
JSON error
final JSON error
EEUU ó Alemania
JSON error
final JSON error
Austria or Germany 
JSON error
final JSON error
Estadounidense
JSON error
final JSON error
Ucrania
JSON error
final JSON error
Mexican
JSON error
final JSON error
None at the moment
JSON error
final JSON error
Germany, United States, London
JSON error
final JSON error
Francophone country
JSON error
final JSON error
AFRICA
JSON error
final JSON error
cualquier lugar
JSON error
final JSON error
Austria 
JSON error
final JSON error
Scotland
JSON error
final JSON error
Alemania, 
JSON error
final JSON error
International/Germany
JSON error
final JSON error
United States m
JSON error
final JSON error
United S

JSON error
final JSON error
Spanish
JSON error
final JSON error
Cuban
JSON error
final JSON error
USA , England ,DUBAI
JSON error
final JSON error
Spojené státy 
JSON error
final JSON error
španělsko, USA, Kanada
JSON error
final JSON error
Portoriko
JSON error
final JSON error
Nezobrazená otázka
JSON error
final JSON error
Švýcarsko 
JSON error
final JSON error
NEPALESE
JSON error
final JSON error
Rakousko 
JSON error
final JSON error
Canad
JSON error
final JSON error
Dansko
JSON error
final JSON error
Skandinavské státy
JSON error
final JSON error
USA,
JSON error
final JSON error
 Portugalsko
JSON error
final JSON error
no lo sé
JSON error
final JSON error
Lincoln Academy, Maine
JSON error
final JSON error
Gruzie
JSON error
final JSON error
Anlige
JSON error
final JSON error
Německé
JSON error
final JSON error
Spojené státy americké 
JSON error
final JSON error
Irska
JSON error
final JSON error
J
JSON error
final JSON error
Česká republika
JSON error
final JSON error
Poľsko
JSON erro

ConnectionError: ('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')"))

In [42]:
df = pd.read_excel('data_migration_030420.xlsx')
#citizenship: D044_01
#df1 = df[['CASE','D044_01','A017_02a','A017_03a','A026_02a','A026_03a','A020_02','A028_02','A010_3a','A022_01','B003_01','B004_01','B005_01','B006_01','B007_01','B008_01','B009_05', 
#          'C011_05','C012_05','C003_09a','C004_09a','C005_14a','D044_01','D045_01','D046_01','D047_01','D049_01','D072_01','D075_01', 'D051_01', 'D052_01',' D053_01','D055_06',
#          'D055_07a','D055_08a','D059_06a','D059_07a','D059_08a','E037_01','E039_01','E027_04a','E027_05a','E027_06a','E028_04a','E028_05a','E028_06a','F005_16a','F007_16a','F008_16a','F018_02','F022_01,'F023_01','F024_01',
#          'E044_01', 'E044_02', 'E044_03', 'E047_01', 'E047_02', 'E048_01, 'E048_02', 'E049_01', 'E049_02', 'E050_01', 'E050_02','E035_01'
#          
#         ]]

#ONLY SECTION D
df1 = df[['CASE','D044_01','D045_01','D046_01','D047_01','D049_01','D072_01','D075_01', 'D051_01', 'D052_01','D053_01','D055_06a','D055_07a','D055_08a','D059_06a','D059_07a','D059_08a'
         ]]
df2=df1[df1.CASE != 'Interview number (ongoing)']


In [62]:
len(list_rows)


1

In [279]:

translate_urls = ["translate.google.com", "translate.google.co.kr",
                      "translate.google.at", "translate.google.de",
                      "translate.google.ru", "translate.google.ch",
                      "translate.google.fr", "translate.google.es"]
tr = Translator(service_urls=translate_urls)


In [56]:
df_translate=pd.read_excel('Countries_Citizenships.xlsx',sheet_name='Tabelle3')  

In [65]:
df_translate.head(n=10)

,COUNTRIES_CITIZENSHIPS
0,Italy
1,United states of America
2,United kingdom
3,Germany
4,USA
5,Sweden
6,Netherlands
7,Australia
8,Singapore
9,United Kingdom
